<a href="https://colab.research.google.com/github/javajaewook/Natural_language_processing/blob/master/Playlist_Maker_With_Doc2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 임포트

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 92kB 11.4MB/s 
     |████████████████████████████████| 460kB 47.5MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

import gensim
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
from gensim import corpora, models
from gensim.models import LdaMulticore
from gensim.models import Word2Vec
from gensim.corpora import Dictionary, MmCorpus

# 데이터 로드

In [ ]:
# 데이터가 있는 폴더로 이동
%cd /content/drive/My Drive/Colab Notebooks/September/data/
%ls

/content/drive/My Drive/Colab Notebooks/September/data
1992~2020_kpop.csv   all_sentences_for_word2vec.txt  melon_crawl_toprated.tsv
all_bigram_model     appos.py                        melon_crawl.tsv
all_lyrics_text.txt  melon_crawl.csv                 model.doc2vec


In [ ]:
df = pd.read_csv("./1992~2020_kpop.csv", engine='python',encoding='CP949')
df.head(1)

,년도,가수,제목,성별,장르,최고순위,작사,작곡,소속사,가사
0,19920000,잼,난 멈추지 않는다,mixed,댄스,1,조진호,조진호,DSP 미디어,이제 모든걸 다시 시작해 내겐 아직도 시간이 있어 때론 상처가 좌절로 남아 돌이킬수...


# 데이터 정제

In [ ]:
len(df['가사'].unique())

5629

In [ ]:
# 가사가 중복되는 행을 제거
df.drop_duplicates(subset=['가사'], inplace=True)

가사가 중복되는 행을 제거했다

In [ ]:
print('총 샘플의 수: ', len(df))

총 샘플의 수:  5629


생각보다 꽤 많이 줄었다<br>
따로 확인해 볼 필요가 있을 것 같다



In [ ]:
df.drop(df[df['장르']=="J-POP"].index, inplace=True)

In [ ]:
print('총 샘플의 수: ', len(df))

총 샘플의 수:  5368


In [ ]:
df.drop(df[df['장르']=="월드뮤직, 중국음악"].index, inplace=True)

In [ ]:
print('총 샘플의 수: ', len(df))

총 샘플의 수:  5360


In [ ]:
df.drop(df[df['가사']=="EMPTY"].index, inplace=True)

In [ ]:
print('총 샘플의 수: ', len(df))

총 샘플의 수:  5359


In [ ]:
len(df['제목', '가수'].values.unique())

4560

당장 처리가 어려운 J-POP과 중국 음악을 데이터프레임에서 제거한다

# 데이터 전처리

In [ ]:
def preprocessing(text):
    # 개행문자 제거
    text =  str(text).strip('\t\n\r')
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', str(text))
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', str(text))
    return text

In [ ]:
sample_lyrics = df.iloc[100]['가사']
type(sample_lyrics)

str

In [ ]:
df.iloc[100]

년도                                               19961100
가수                                                    노이즈
제목                                             착각 (Remix)
성별                                                   male
장르                                                     댄스
최고순위                                                    0
작사                                                    김창환
작곡                                                    김창환
소속사                                                 라인 기획
가사      너의 친절함에 나는 속았었나봐 너도 날 좋아한다고 그렇게 믿었는데 너의 성격이 원래...
Name: 102, dtype: object

In [ ]:
print(sample_lyrics)

너의 친절함에 나는 속았었나봐 너도 날 좋아한다고 그렇게 믿었는데 너의 성격이 원래 상냥하다는 걸 알면서도 나는 왠지 그렇게 믿었었어 그러던 어느 날인가 너는 나에게 너의 애인을 소개시켜주며 인사를 나누라는데 나의 눈앞이 왜 자꾸 캄캄해져만 오는지 어색해하는 나의 표정을 바라보며 너두 그제서야 내가 널 좋아하는 걸 알았나봐 하지만 그래도 어쩔 수가 없잖아 그저 나혼자의 착각속에 너를 좋아한것 뿐이니까 너를 보면서 난 착각을 해왔었지 너의 친절함을 나는 믿었었기에 그게 사랑인줄 알고 나는 혼자서 고민해왔었j어 너의 변함없는 태도를 보면서 나는 조금은 이상하다는 생각을 했지만 그래도 난 내 자신을 믿고 싶었어 니가 날 좋아한다고 그러던 어느 날인가 너는 나에게 너의 애인을 소개시켜주며 인사를 나누라는데 나의 눈앞이 왜 자꾸 캄캄해져만 오는지 어색해하는 나의 표정을 바라보며 너두 그제서야 내가 널 좋아하는 걸 알았나봐 하지만 그래도 어쩔 수가 없잖아 그저 나혼자의 착각속에 너를 좋아한것 뿐이니까 - 간 주 중 - 너의 친절함에 나는 속았었나봐 너도 날 좋아한다고 그렇게 믿었는데 너의 성격이 원래 상냥하다는 걸 알면서도 나는 왠지 그렇게 믿었었어 그러던 어느 날인가 너는 나에게 너의 애인을 소개시켜주며 인사를 나누라는데 나의 눈앞이 왜 자꾸 캄캄해져만 오는지 어색해하는 나의 표정을 바라보며 너두 그제서야 내가 널 좋아하는 걸 알았나봐 하지만 그래도 어쩔 수가 없잖아 그저 나혼자의 착각속에 너를 좋아한것 뿐이니까 어색해하는 나의 표정을 바라보며 너두 그제서야 내가 널 좋아하는 걸 알았나봐 하지만 그래도 어쩔 수가 없잖아 그저 나혼자의 착각속에 너를 좋아한것 뿐이니까


전처리 함수가 잘 작동하는지 확인하기 위해 샘플 가사를 처리해보자

In [ ]:
sample_lyrics = preprocessing(sample_lyrics)
sample_lyrics[:200]

'너의 친절함에 나는 속았었나봐 너도 날 좋아한다고 그렇게 믿었는데 너의 성격이 원래 상냥하다는 걸 알면서도 나는 왠지 그렇게 믿었었어 그러던 어느 날인가 너는 나에게 너의 애인을 소개시켜주며 인사를 나누라는데 나의 눈앞이 왜 자꾸 캄캄해져만 오는지 어색해하는 나의 표정을 바라보며 너두 그제서야 내가 널 좋아하는 걸 알았나봐 하지만 그래도 어쩔 수가 없잖아 '

잘 되는 것 같으니 전체 데이터에 대해 적용한다

In [ ]:
%time lyrics = df['가사'].apply(preprocessing)

CPU times: user 624 ms, sys: 3.4 ms, total: 627 ms
Wall time: 634 ms


In [ ]:
lyrics.head()

0    이제 모든걸 다시 시작해 내겐 아직도 시간이 있어 때론 상처가 좌절로 남아 돌이킬수...
1    워 우리 모두 사랑하자 우리의 젊은날을 위하여 우리 모두 춤을 추자 가벼운 인스텝 ...
2    어둠은 드리워지고 이제는 우리들만의 시간이 지친듯이 내 가슴속에 밀려와 나를 재우고...
3    끝없는 어둠 속을 천천히 걷고 있어 어디서 본 듯한 야릇한 느낌이야 거리는 비에 젖...
4    요즘 친구들은 흥 정말 진정한 친구가 뭔지 잘 몰라 말로만 떠들고 자기들이 다만 그...
Name: 가사, dtype: object

In [ ]:
titles = df['제목']
groups = df['가수']
titles_groups = pd.concat([titles, groups],axis=1)

In [ ]:
titles_groups.head(5)

,제목,가수
0,난 멈추지 않는다,잼
1,우리 모두 사랑하자,잼
2,이유,잼
3,18번가의 비밀,잼
4,요즘 친구들,잼


# 토큰화

In [ ]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
okt = Okt()

In [ ]:
lyrics_tokens = []
for lyric in lyrics:
    temp_token = []
    temp_token = okt.morphs(lyric, stem=True) # 토큰화
    temp_token = [word for word in temp_token if not word in stopwords]
    lyrics_tokens.append(temp_token)

In [ ]:
import re
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

def preprocessing(text):
    text =  str(text).strip('\t\n\r')
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', str(text))
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', str(text))
    return text

def testData_preprocess(text):
    temp = preprocessing(text)
    temp_token = []
    temp_token = okt.morphs(temp, stem=True) # 토큰화
    temp_token = [word for word in temp_token if not word in stopwords]
    return temp_token

In [ ]:
lyrics_tokens[0][0:10]

['이제', '모든', '걸', '다시', '시작', '내겐', '아직도', '시간', '있다', '때론']

In [ ]:
len(titles)

5359

In [ ]:
len(lyrics_tokens)

5359

# Doc2Vec

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
def makeTaggedDocument(base_document):
    documents = []
    # df['가수'].iloc[i]
    for i, doc in enumerate(base_document):
        taggedDoc = TaggedDocument(words=doc, tags=[df['제목'].iloc[i]])
        documents.append(taggedDoc)
    
    return documents

In [ ]:
documents = makeTaggedDocument(lyrics_tokens)

In [ ]:
documents[2]

TaggedDocument(words=['어둠', '드리워지다', '이제', '우리', '만의', '시간', '지치다', '내', '가슴속', '밀려오다', '나르다', '재우다', '헝클어지다', '너', '긴머리', '향', '긋다', '향기', '잠시', '취해', '서', '모든', '것', '잊어버리다', '채', '어둠', '끝', '떠나다', '얼마나', '시간', '흐르다', '흐려지다', '너', '눈', '애처롭다', '보이다', '이유', '없이', '사랑', '말', '그대', '나르다', '이해', '있다', '한동안', '아무', '말', '없이', '그대', '흩어지다', '나르다', '보다', '머뭇거리다', '그냥', '울', '고', '말다', '헝클어지다', '너', '긴머리', '향', '긋다', '향기', '잠시', '취해', '서', '모든', '것', '잊어버리다', '채', '어둠', '끝', '떠나다', '얼마나', '시간', '흐르다', '흐려지다', '너', '눈', '애처롭다', '보이다', '이유', '없이', '사랑', '말', '그대', '나르다', '이해', '있다', '한동안', '아무', '말', '없이', '그대', '흩어지다', '나르다', '보다', '머뭇거리다', '그냥', '울', '고', '말다'], tags=['이유'])

In [ ]:
setting = {
    'dm' : 1,
    'embedding_size' : 100,
    'epochs' : 30,
    'window' : 2,
    'min_count' : 0,
    'workers' : 4,
}

In [ ]:
len(documents)

5359

In [ ]:
model = Doc2Vec(documents, 
                vector_size = setting['embedding_size'],
                epochs = setting['epochs'],
                window = setting['window'],
                min_count = setting['min_count'],
                workers = setting['workers'],
                divmod = setting['dm'],
                negative=5)

In [ ]:
model.most_similar('사랑')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('이별', 0.5642892718315125),
 ('꿈', 0.5222277045249939),
 ('행복', 0.5205470323562622),
 ('말', 0.5184621810913086),
 ('세상', 0.5156251192092896),
 ('좋아하다', 0.5101331472396851),
 ('믿다', 0.5089309215545654),
 ('원하다', 0.5053156614303589),
 ('몫', 0.5015321373939514),
 ('어쩌면', 0.49567991495132446)]

In [ ]:
len(model.docvecs)

4560

In [ ]:
model.docvecs.doctags

{'난 멈추지 않는다': Doctag(offset=0, word_count=107, doc_count=1),
 '우리 모두 사랑하자': Doctag(offset=1, word_count=180, doc_count=1),
 '이유': Doctag(offset=2, word_count=559, doc_count=4),
 '18번가의 비밀': Doctag(offset=3, word_count=143, doc_count=1),
 '요즘 친구들': Doctag(offset=4, word_count=155, doc_count=1),
 '싫증': Doctag(offset=5, word_count=242, doc_count=1),
 '난 알아요': Doctag(offset=6, word_count=196, doc_count=1),
 '이밤이 깊어가지만': Doctag(offset=7, word_count=137, doc_count=1),
 '너와 함께한 시간속에서': Doctag(offset=8, word_count=129, doc_count=1),
 '너에게 원한건': Doctag(offset=9, word_count=317, doc_count=2),
 '변명': Doctag(offset=10, word_count=531, doc_count=2),
 '나를 돌아봐': Doctag(offset=11, word_count=214, doc_count=1),
 '하여가(何如歌)': Doctag(offset=12, word_count=337, doc_count=1),
 '마지막 축제': Doctag(offset=13, word_count=195, doc_count=1),
 '너에게': Doctag(offset=14, word_count=286, doc_count=2),
 '어색한 느낌': Doctag(offset=15, word_count=103, doc_count=1),
 '잼 (Zam) 우리는': Doctag(offset=16, word_count=92, doc_count=1)

In [ ]:
model.save('../model/model2.doc2vec')

In [ ]:
model.docvecs.most_similar(positive=['활주'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('활주 (`나루토` 여는 노래)', 0.9094610214233398),
 ('날개를 펴고', 0.606087327003479),
 ('역전', 0.5801390409469604),
 ('Phantom 환영 (幻影)', 0.5569276809692383),
 ('Phantom - 환영 (幻影)', 0.5523754358291626),
 ('Non-Stop', 0.552303671836853),
 ('ILLUSION (Arabic Version)', 0.5436040759086609),
 ('우리는', 0.5391476154327393),
 ('졸린 꿈', 0.5350289344787598),
 ('STILL ALIVE', 0.5283910036087036)]

In [ ]:
test = '내게 허락된 건 힘겹기만한 거친 미래라해도 나를 깨운 꿈엔 모든 걸 걸고 달려갈꺼야! 나뭇잎 끝에 흩어지는 바람에 나의 꿈이 흔들려 하지만 아직 내게 깃든 운명을 탓할 필요는 없지 모두가 날 알아줄 그날이 언젠가 찾아올테니 누군가 앞서 달린 이 길을 가네 (Yeah-!) 내게 허락된 건 힘겹기만 거친 미래라 해도 나를 깨운 꿈에 모든 걸 걸고 달려갈거야. 헛된 일이라고 말하지는 마 나는 남들과 달라 모든 힘을 다해 이뤄낼거야 나를 지켜봐(워!)'
testtoken = testData_preprocess(test)
testvector = model.infer_vector(testtoken)

In [ ]:
model.docvecs.most_similar(positive=[testvector], topn=10)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('활주 (`나루토` 여는 노래)', 0.8435726165771484),
 ('활주', 0.8323813676834106),
 ('Phantom 환영 (幻影)', 0.576973021030426),
 ('Phantom - 환영 (幻影)', 0.5760331153869629),
 ('날개를 펴고', 0.5549038648605347),
 ('BIG TRANSFORMER', 0.5268539786338806),
 ('Success', 0.5004888772964478),
 ('축제 (Carnival)', 0.48383989930152893),
 ('연습생', 0.4811514616012573),
 ('I`M OK', 0.48073872923851013)]

In [ ]:
vector_docs = model.docvecs.vectors_docs

In [ ]:
vector_docs = model.docvecs.vectors_docs
doctags = []
for i, docvec in enumerate(vector_docs):
    tag = model.docvecs.index_to_doctag(i)
    doctags.append([tag, docvec])

In [ ]:
doctags[:10]

[['난 멈추지 않는다',
  array([ 0.28568727,  0.38717416,  0.03774367, -0.7994696 , -0.84676325,
         -0.34828785,  0.86713314,  0.08077856, -1.1671879 ,  0.13303354,
         -1.224377  ,  0.16945402,  1.447298  , -0.65575045,  0.5375399 ,
          1.0250323 ,  0.3735826 , -0.01137778, -0.22659442,  0.4737745 ,
         -1.072029  ,  0.97746855, -0.28885633,  0.49423364, -0.6381586 ,
         -0.49857566, -0.15322435, -0.3640548 ,  1.5934478 , -0.33667776,
         -0.14397511, -0.47045407, -0.85029095,  0.95856166,  0.5604979 ,
          0.57064915, -0.40429235, -0.04138876,  1.2928578 ,  0.552922  ,
         -0.31140602,  1.1917244 , -0.08265329,  0.07500198,  0.75792754,
         -1.2951127 , -0.3605851 , -0.1474624 , -0.39712203,  0.31023937,
          0.18160494, -0.87745625,  1.284456  ,  1.0776645 ,  0.0776516 ,
          0.24841964,  0.23260924, -0.23152228, -0.36651796,  0.31870183,
         -0.7892106 ,  1.0758613 ,  0.3760328 ,  0.05110176, -0.80149204,
         -0.46577102, -

In [ ]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

In [ ]:
def get_most_similar(docvec, topn=10):
    tag_similarity = []

    for i in range(len(doctags)):
        tag_similarity.append([doctags[i][0], cos_sim(docvec, doctags[i][1])])

    sorted_by_sim = sorted(tag_similarity, 
                           key=lambda tag_similarity: tag_similarity[1], 
                           reverse=True)

    return sorted_by_sim[:topn]

In [ ]:
def get_title_1(title, topn=10): # 제목으로 가져오기
    result = []
    try: # 입력한 제목이 기존 태그에 있을 경우
        result = get_most_similar(model.docvecs[text], topn=topn)
    except: # 입력한 제목이 기존 태그에 없을 경우 (결과의 성능 보장 X)
        result = get_most_similar(model.infer_vector(title), topn=topn)

    return result

In [ ]:
def get_title_2(text, topn=10): # 문장으로 가져오기
    tokens = testData_preprocess(text)
    infered_vec = model.infer_vector(tokens)
    result = get_most_similar(infered_vec, topn)
    return result

In [ ]:
print(get_title_2(test))

[['활주 (`나루토` 여는 노래)', 0.8371808], ['활주', 0.83224785], ['Phantom 환영 (幻影)', 0.5715971], ['Phantom - 환영 (幻影)', 0.5677924], ['날개를 펴고', 0.5646731], ['BIG TRANSFORMER', 0.520629], ['Success', 0.51087284], ['나비잠 (Sweet Dream)', 0.501549], ['I`M OK', 0.49688733], ['Born To Fight', 0.4945453]]


In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

%matplotlib inline
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [ ]:
def tsne_plot():
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for i in range(len(doctags)):
        tokens.append(doctags[i][1])
        labels.append(doctags[i][0])
    
    tsne_model = TSNE(perplexity=40, n_components=2, n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(10, 10)) 
    for i in range(len(x)):
        try:
            plt.scatter(x[i],y[i])
            plt.annotate(labels[i],
                        xy=(x[i], y[i]),
                        xytext=(5, 2),
                        textcoords='offset points',
                        ha='right',
                        va='bottom')
        except:
            continue
    plt.show()

In [ ]:
tsne_plot()

/usr/local/lib/python3.6/dist-packages/matplotlib/backends/backend_agg.py:214: RuntimeWarning: Glyph 45212 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/usr/local/lib/python3.6/dist-packages/matplotlib/backends/backend_agg.py:214: RuntimeWarning: Glyph 47688 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/usr/local/lib/python3.6/dist-packages/matplotlib/backends/backend_agg.py:214: RuntimeWarning: Glyph 52628 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/usr/local/lib/python3.6/dist-packages/matplotlib/backends/backend_agg.py:214: RuntimeWarning: Glyph 51648 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/usr/local/lib/python3.6/dist-packages/matplotlib/backends/backend_agg.py:214: RuntimeWarning: Glyph 50506 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/usr/local/lib/python3.6/dist-packages/matplotlib/backends/backend_agg.py:214: RuntimeWarning: Glyph 45716 missing from current font.
  font.set_

ValueError: ignored

<Figure size 720x720 with 1 Axes>